In [1]:
import json
import os
from os.path import join as pathj
root_dir = r'D:\sa_data\f36_exps\20221124_aware_hardnms_conf10_iou50\Element_grabbing\grabbing_evaluation\labelme\grabbing_evaluation_statistics\statistic_0.5_0.5_0.5_0\detect'
miss = 'button_miss' #有label
fault = 'button_fault'
miss_dir = os.path.join(root_dir,miss)
fault_dir = os.path.join(root_dir,fault)
# 读取json
def read_json(abs_txt_name):
    with open(abs_txt_name, 'r', encoding='utf-8') as f: #读文件名
        ret_dic = json.load(f)
        return ret_dic
        
def save_json(new_name,json_ins):
    with open(new_name, 'w',encoding='utf-8') as f:
        json.dump(json_ins ,f ,indent=2,ensure_ascii=False)#zw
        print('{} saved'.format(new_name)) 

# 1.漏报和误报分析

In [5]:
fault_dir = r'C:\Users\tingfeng-wu\Desktop\02_github\py_script\notebook\linebox_fault'
fault_list =[]

for name in os.listdir(fault_dir):
    instance = read_json(pathj(fault_dir, name))
    linebox_miss = instance['linebox_fault']
    if(len(linebox_miss) == 0): 
        continue
    for e in linebox_miss:
        fault_list.append(e['box'])

In [8]:
thin_num1 = 0
zero_count =0
for e in fault_list: #误报
    w = abs(e[2] -e[0])
    h = abs(e[3] -e[1])
    try:
        ratio = max(w/h, h/w)
        min_p = min(w,h)
    except:
        # print(f'w:{w} h:{h}')
        # print(f'bbox:{e}')
        zero_count+=1
        continue
    if min_p < 8 :
    # if ratio>10 and ratio <20:
        thin_num1 +=1
print(thin_num1/(len(fault_list)-zero_count))

print(f'{len(fault_list)-zero_count}')
print(f"zero:{zero_count},percent:{zero_count/len(fault_list)}")

0.656773751000343
43735
zero:1142,percent:0.025447333823562183


In [2]:
miss_list =[]
for name in os.listdir(miss_dir):#漏报
    instance = read_json(pathj(miss_dir, name))
    linebox_miss = instance['linebox']
    if(len(linebox_miss) == 0): 
        continue
    for e in linebox_miss:
        miss_list.append(e['location'])


In [9]:
thin_num = 0
for e in miss_list:
    w = abs(e[2] -e[0])
    h = abs(e[3] -e[1])
    min_p = min(w,h)
    ratio = max(w/h, h/w)
    # if ratio <10:
    if min_p < 8:
        thin_num+=1
print(thin_num/len(miss_list))


0.922692036645525


In [24]:
print(f'漏报数目：{len(miss_list)}')
miss_len = len(miss_list)
count_smaller_1 = 0
count_smaller_10 = 0
count_smaller_20 = 0
count_smaller_50 = 0
count_greater_50 = 0
for a in miss_list:
    w = abs(a[2] -a[0])
    h = abs(a[3] -a[1])
    ratio = max(w/h, h/w)
    if(ratio<1):
        count_smaller_1+=1
    elif(ratio<10):
        count_smaller_10+=1
    elif(ratio<20):
        count_smaller_20+=1
    elif(ratio<50):
        count_smaller_50+=1
    else:
        count_greater_50+=1
print(f'ratio<1 | ratio<10 | ratio<20 | ratio<50 | ratio>50')
print(f'{count_smaller_1}\{count_smaller_10}\{count_smaller_20}\{count_smaller_50}\{count_greater_50}')
print(f'{count_smaller_1/miss_len}\{count_smaller_10/miss_len}\{count_smaller_20/miss_len}\{count_smaller_50/miss_len}\{count_greater_50/miss_len}')

漏报数目：14190
ratio<1 | ratio<10 | ratio<20 | ratio<50 | ratio>50
0\4333\2075\3690\4092
0.0\0.30535588442565187\0.14622973925299507\0.26004228329809725\0.28837209302325584


## 2.labelme 2 sa_subcate 统计labelme的所有类别

In [21]:
# 'D:\sa_data\sa_coco_1\labelme\labelme_json'
dir = r'D:\sa_data\0011_v1.6_filtered_linebox_yolox\labelme'
names =[x for x in  os.listdir(dir) if x.endswith('json')]
cls = set()
for name in names:
    ins = read_json(pathj(dir, name))
    for one in ins['shapes']:
        if len(one['label']) > 15:
            continue
        else:
            cls.add(one['label'])

In [20]:

with open('1.6.1.txt', mode='w') as f:
    for i in cls:
        f.write(i+'\n')

In [3]:
cls_name = list(cls)
cls_3 =set(['icon','text','image'])
cls_name_need = list(cls - cls_3)
print(cls_name_need)

['h_segline', 'v_segline', 'tab', 'icon_text', 'linebox', 'track', 'mohu', 'inputbox', 'scrollbar_slider']


## 3.各个细类别的recall

In [22]:
dir = 'D:\sa_data\sa_coco_1\labelme\comps_linebox_subcate'
names = os.listdir(dir)
cls_name_need = ['inputbox', 'v_segline', 'tab', 'track', 
    'linebox', 'h_segline', 'scrollbar_slider']

gold_dict ={} #每个类别gt的数量
for name in cls_name_need:
    gold_dict[name] = 0
# cls = set()
for name in names:
    ins = read_json(pathj(dir, name))
    for name in cls_name_need:
        gold_dict[name] += len(ins[name])
        # cls.add(one['label'])

In [15]:
predicts = r'D:\sa_data\f36_exps\20221124_aware_hardnms_conf10_iou50\Element_grabbing\grabbing_evaluation\labelme\grabbing_predict'
need_name  = ['linebox']
pred_bboxes =[]
for pred in os.listdir(predicts):
    pred_ins = read_json(pathj(predicts, pred))
    for name in need_name:
        try:
            if(len(pred_ins[name])==0):
                continue
            else:
                for box_dict in pred_ins[name]:
                    pred_bboxes.append(box_dict["location"])
        except:
            print('KeyError')
        
print(json.dumps(gold_dict ,indent=4, sort_keys=True))
gold_dict.items()
# print(sum(gold_dict.items()))
print(len(pred_bboxes))

In [23]:
def iou(a, b):
    # 计算两个检测框之间的iou
    w_a = a[2] - a[0]
    h_a = a[3] - a[1]
    w_b = b[2] - b[0]
    h_b = b[3] - b[1]

    area_a = w_a * h_a
    area_b = w_b * h_b

    w = min(b[2], a[2]) - max(b[0], a[0])
    h = min(b[3], a[3]) - max(b[1], a[1])

    if w <= 0 or h <= 0:
        return 0

    area_c = w * h

    return area_c / (area_a + area_b - area_c)

import numpy as np
def iou_np(boxes0: np.ndarray, boxes1: np.ndarray):
    """ 计算多个边界框和多个边界框的交并比

    Parameters
    ----------
    boxes0: `~np.ndarray` of shape `(A, 4)`
        边界框

    boxes1: `~np.ndarray` of shape `(B, 4)`
        边界框

    Returns
    -------
    iou: `~np.ndarray` of shape `(A, B)`
        交并比
    """
    A = boxes0.shape[0]
    B = boxes1.shape[0]

    xy_max = np.minimum(boxes0[:, np.newaxis, 2:].repeat(B, axis=1),
                        np.broadcast_to(boxes1[:, 2:], (A, B, 2)))
    xy_min = np.maximum(boxes0[:, np.newaxis, :2].repeat(B, axis=1),
                        np.broadcast_to(boxes1[:, :2], (A, B, 2)))

    # 计算交集面积
    inter = np.clip(xy_max-xy_min, a_min=0, a_max=np.inf)
    inter = inter[:, :, 0]*inter[:, :, 1]

    # 计算每个矩阵的面积
    area_0 = ((boxes0[:, 2]-boxes0[:, 0])*(
        boxes0[:, 3] - boxes0[:, 1]))[:, np.newaxis].repeat(B, axis=1)
    area_1 = ((boxes1[:, 2] - boxes1[:, 0])*(
        boxes1[:, 3] - boxes1[:, 1]))[np.newaxis, :].repeat(A, axis=0)

    return inter/(area_0+area_1-inter)

def get_iou_thresh(a):
    # 文本采用动态iou，最大可以到MAX_IOU，对于小目标最小iou可以到0.1。
    w_a = a[2] - a[0]
    h_a = a[3] - a[1]
    area = (w_a + 1) * (h_a + 1)
    iou_thresh = min((area+50)/500, 0.5)
    return iou_thresh

In [24]:
# start
dir = 'D:\sa_data\sa_coco_1\labelme\comps_linebox_subcate'
predicts = r'D:\sa_data\f36_exps\20221124_aware_hardnms_conf10_iou50\Element_grabbing\grabbing_evaluation\labelme\grabbing_predict'

In [4]:
cls_name_need = ['inputbox', 'v_segline', 'tab', 'track', 
    'linebox', 'h_segline', 'scrollbar_slider']
tps_dict = {}
gold_dict ={} #每个类别gt的数量
pred_dict ={'linebox_predict':0}
for name in cls_name_need:
    gold_dict[name] = 0
for name in cls_name_need:
    tps_dict[name+'_tp'] = 0

for name in os.listdir(dir): #每个文件进行对比
    pred_ins = read_json(pathj(predicts, name))
    gold_ins = read_json(pathj(dir, name))
    dt_per_img = []
    gt_per_img = []
    dt = pred_ins['linebox']
    pred_dict['linebox_predict'] += len(dt)
    for e in dt:
        dt_per_img.append({'box':e['location'], 'label': 'linebox'})
    for name in cls_name_need:
        gt = gold_ins[name]
        # gold_dict[name] += len(ins[name])
        for g in gt:
            gt_per_img.append({'box':g['location'], 'label':name})

    #一张图的 dt和gt都有了
    predcit_status = np.zeros(len(dt_per_img), dtype=np.int)
    for gt in gt_per_img:
        dyn_iou = get_iou_thresh(gt['box'])
        iou_thresh = min(0.5, dyn_iou)
        count = -1
        for dt in dt_per_img:
            count+=1
            if predcit_status[count] == 0:
                cur_iou = iou(gt['box'], dt['box'])
                cur_gt_label = gt['label']
                if cur_iou > iou_thresh:
                    tps_dict[cur_gt_label+'_tp']+=1
                    predcit_status[count] = 1
                    break
                else:
                    continue
            else: 
                continue

<ipython-input-4-1b9810f1eee0>:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predcit_status = np.zeros(len(dt_per_img), dtype=np.int)


In [6]:
print(json.dumps(tps_dict, indent=4,sort_keys=True))
print(json.dumps(gold_dict ,indent=4, sort_keys=True))
print(json.dumps(pred_dict ,indent=4, sort_keys=True))

{
    "h_segline_tp": 3775,
    "inputbox_tp": 14024,
    "linebox_tp": 7339,
    "scrollbar_slider_tp": 6280,
    "tab_tp": 16606,
    "track_tp": 3324,
    "v_segline_tp": 19269
}
{
    "h_segline": 0,
    "inputbox": 0,
    "linebox": 0,
    "scrollbar_slider": 0,
    "tab": 0,
    "track": 0,
    "v_segline": 0
}
{
    "linebox_predict": 115494
}
